In [1]:
import logging
from lisa.utils import setup_logging
setup_logging()

2018-09-21 17:24:16,841 INFO    : root         : Using LISA logging configuration:
2018-09-21 17:24:16,842 INFO    : root         :   /home/dourai01/Data/Git/lisa/py3k/logging.conf


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from lisa.env import TestEnv, TargetConfig

# Import support for Android devices
from lisa.android import Screen, Workload

# Support for trace events analysis
from lisa.trace import Trace

# Suport for FTrace events parsing and visualization
import trappy
import pandas as pd

from lisa.wlgen.rta import RTA, Periodic, Ramp
from time import sleep

from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


In [4]:
linux_hikey = {
    "platform" : "linux",
    "board" : "hikey",
    "host": "pwrsft-hikey620-1",
    #"username" : "root",
    #"password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"],
    "rtapp-calib" :  {
        "0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136
     }
}

In [5]:
linux_juno = {
    "platform" : "linux",
    "board" : "juno",
    "host": "192.168.0.1",
    "username" : "root",
    "password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [6]:
android_hikey = {
    "platform" : "android",
    "board" : "hikey960",
    "modules" : ["sched", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [7]:
te = TestEnv(TargetConfig(linux_hikey))
target = te.target

2018-09-21 17:24:17,730 INFO    : lisa.env.TestEnv : Using base path: /home/dourai01/Data/Git/lisa/py3k
2018-09-21 17:24:17,731 INFO    : lisa.env.TestEnv : Devlib modules to load: ['hotplug', 'sched', 'cpufreq', 'cpuidle', 'cgroups']
2018-09-21 17:24:17,732 INFO    : lisa.env.TestEnv : Connecting linux target:
2018-09-21 17:24:17,733 INFO    : lisa.env.TestEnv :   username : root
2018-09-21 17:24:17,733 INFO    : lisa.env.TestEnv :   password : 
2018-09-21 17:24:17,734 INFO    : lisa.env.TestEnv :       host : pwrsft-hikey620-1
2018-09-21 17:24:17,734 INFO    : lisa.env.TestEnv : Connection settings:
2018-09-21 17:24:17,734 INFO    : lisa.env.TestEnv :    {'username': 'root', 'password': '', 'host': 'pwrsft-hikey620-1'}
2018-09-21 17:24:23,869 INFO    : lisa.env.TestEnv : Initializing target workdir:
2018-09-21 17:24:23,870 INFO    : lisa.env.TestEnv :    /root/devlib-target
2018-09-21 17:24:24,844 INFO    : CGroups      : Available controllers:
2018-09-21 17:24:25,454 INFO    : CGrou

### Generics

In [8]:
from lisa.tests.kernel.scheduler.eas_behaviour import *

In [9]:
tests = [OneSmallTask]#, ThreeSmallTasks, TwoBigTasks, TwoBigThreeSmall, RampUp, RampDown, EnergyModelWakeMigration]

In [10]:
res = {}

for test in tests:
    bundle = test.from_target(te)
    res[test.__name__] = bundle

2018-09-21 17:24:37,770 INFO    : lisa.env.TestEnv : Using configuration provided RTApp calibration
2018-09-21 17:24:37,772 INFO    : lisa.env.TestEnv : Using RT-App calibration values:
2018-09-21 17:24:37,773 INFO    : lisa.env.TestEnv :    {"0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136}
2018-09-21 17:24:38,164 INFO    : lisa.wlgen.rta.RTA : Calibration value: 136
2018-09-21 17:24:38,164 INFO    : lisa.wlgen.rta.RTA : Default policy: SCHED_OTHER
2018-09-21 17:24:38,165 INFO    : lisa.wlgen.rta.RTA : ------------------------
2018-09-21 17:24:38,165 INFO    : lisa.wlgen.rta.RTA : task [small], sched: using default policy
2018-09-21 17:24:38,166 INFO    : lisa.wlgen.rta.RTA :  | start delay: 0.000000 [s]
2018-09-21 17:24:38,166 INFO    : lisa.wlgen.rta.RTA :  | loops count: 1
2018-09-21 17:24:38,167 INFO    : lisa.wlgen.rta.RTA :  + phase_000001
2018-09-21 17:24:38,167 INFO    : lisa.wlgen.rta.Phase :  | duration 1.000000 [s] (62 loops)
2018-09-21 17:24:3

In [11]:
for test_name, bundle in res.items():
    print(test_name)
    print("---")
    print("test_slack: {}".format(bundle.test_slack()))
    print("test_task_placement: {}".format(bundle.test_task_placement()))
    print()

OneSmallTask
---
test_slack: FAILED: slack_small_0 = 0.0 %, slack_small_1 = 0.0 %, slack_small_2 = 0.0 %, slack_small = 100.0 %


2018-09-21 17:25:02,302 WARNING : lisa.trace.Trace : Kernel version not available from platform data
2018-09-21 17:25:02,303 INFO    : lisa.trace.Trace : Parsing trace assuming kernel v3.18


test_task_placement: PASSED: estimated_energy = 144.34142511858244 bogo-joules, energy_threshold = 245.74051446945342 bogo-joules



### CPU sanity check

In [12]:
from lisa.tests.kernel.misc.capacity_sanity import CapacitySanityCheck

In [13]:
bundle = CapacitySanityCheck.from_target(te)

2018-09-21 17:25:07,820 INFO    : lisa.wlgen.sysbench.Sysbench : Execution start: /root/devlib-target/bin/taskset 0x1 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-21 17:25:09,002 INFO    : lisa.wlgen.sysbench.Sysbench : Execution complete
2018-09-21 17:25:09,337 INFO    : lisa.wlgen.sysbench.Sysbench : Execution start: /root/devlib-target/bin/taskset 0x2 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-21 17:25:10,516 INFO    : lisa.wlgen.sysbench.Sysbench : Execution complete
2018-09-21 17:25:10,850 INFO    : lisa.wlgen.sysbench.Sysbench : Execution start: /root/devlib-target/bin/taskset 0x4 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-21 17:25:12,027 INFO    : lisa.wlgen.sysbench.Sysbench : Execution complete
2018-09-21 17:25:12,361 INFO    : lisa.wlgen.sysbench.Sysbench : Execution start: /root/devlib-target/bin/taskset 0x8 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-21 17:25:13,539 INFO    

In [14]:
print(str(bundle.test_capacity_sanity()))

PASSED: Performance @1024 capacity = 227


### Hotplug

In [15]:
from lisa.tests.kernel.hotplug.torture import HotplugTorture

In [16]:
bundle = HotplugTorture.from_target(te)

In [20]:
print(bundle.test_cpus_alive().result.name)

PASSED


In [18]:
print(bundle.test_cpus_alive().metrics)

{'hotpluggable CPUs': TestMetric([0, 1, 2, 3, 4, 5, 6, 7], None), 'Online CPUs': TestMetric([0, 1, 2, 3, 4, 5, 6, 7], None)}


In [21]:
print(bundle.test_target_alive().result.name)

PASSED
